## Creative Optimization Tutorial

### What resources do we have

Resources:
- Creative URL
- Assets
- Performance Score

#### Where to Begin ??

- Need to find a way to represent a creative or parts of it
  - Extracting:
    - Start Frame and
    - End Frame
  - Extracting the entire creative as a video

#### Extracting Start Frame and End Frame or Preview Video of a Creative

> We can use Selenium to Load the creative on a automated browser follow its progression and capture screenshots at certain moments or Record the entire creative.

In [1]:
# Imports
from typing import Tuple
from time import sleep
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from os import path
from subprocess import Popen, call
import subprocess
import pyautogui
import ffmpeg

In [2]:
# from selenium import webdriver
# from webdriver_manager.chrome import ChromeDriverManager

# # driver = webdriver.Chrome(ChromeDriverManager().install())
# driver = webdriver.Chrome('C:/chromedriver.exe');

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

# driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
# driver.get("https://www.google.com")


In [3]:
class CreativeFrameExtractor:
    '''
    Class responsible for Extracting Creative Start and End Frames.
    It requires a chrome webdriver compatible with selenium to be
    installed/included in the run environment path.
    '''

    def __init__(self, preview_url: str, 
                 engagement_type: str, 
                 save_location: str = '',
                 browser_edges: Tuple[float, float] = (70, 1039)) -> None:
        
        self.preview_url = preview_url
        self.engagement_type = engagement_type
        self.browser_edges = browser_edges
        self.file_name = '-'.join(preview_url.split('/')[-3:-1])
        self.save_location = save_location
        self.video_name = path.join(self.save_location, self.file_name)
        print(self.video_name)
        print(self.file_name)
        print(self.save_location)
        self.cmd = f"ffmpeg -f gdigrab -draw_mouse 0 -framerate 60 -i desktop -vcodec libx264rgb {self.video_name}.mp4 -y"
        # self.cmd = f"ffmpeg -f gdigrab -offset_x 1600 -offset_y 0 -video_size 1600x1200 -draw_mouse 0 -framerate 60 -i desktop -vcodec libx264rgb {self.video_name}.mp4"
        
        # Configurations

        # Browser Configuration
        # Browser Options
        self.opt = Options()
        self.opt.add_argument("--hide-scrollbars")
        self.opt.add_experimental_option(
            "excludeSwitches", ["enable-automation"])
        # Browser Logs
        self.capabilities = DesiredCapabilities.CHROME
        self.capabilities["goog:loggingPrefs"] = {"browser": "ALL"}

    def is_status_complete(self, passed_driver) -> bool:
        '''
        Function to check status of the AD-Unit and its completion.
        '''
        # Retrieve logs from browser
        logs = passed_driver.get_log("browser")

        for log in logs:
            # Select logs coming from AD-Unit
            if log["source"] == "console-api":
                # Extract message from log
                message = log["message"]

                if '"GAME CREATED"' in message or '"DROPPED"' in message:
                    # Start Recording Game
                    print("Starting Recording AD-UNIT...")
                    print(log)
                    return False

                if '"START"' in message:
                    # Engaged
                    print("AD-UNIT Engaged...")
                    print(log)
                    return False

                if '"GAME COMPLETE"' in message:
                    # Stop Recording Game
                    print("Stopped Recording AD-UNIT...")
                    print(log)
                    return True

        return False
    
    @staticmethod
    def terminate(process):
        '''
        Function to stop/terminate a process.
        '''
        # Video Recording Process Terminator
        if process.poll() is None:
            call("taskkill /F /T /PID " + str(process.pid))

    @staticmethod
    def crop_video(filename: str, x_pos: float = 0, y_pos: float = 70, width: float = 650, height: float = 970) -> None:
        '''
        Function to crop a video with given location and size specific parameters.
        '''
        print(filename)
        input_video = ffmpeg.input(f"{filename}.mp4")
        cropped_video = ffmpeg.crop(input_video, x_pos, y_pos, width, height)
        output_video = ffmpeg.output(cropped_video, f"{filename}_cropped.mp4")
        ffmpeg.run(output_video)

    def _imitate_engagement(self, ad_size: Tuple[float, float]) -> None:
        '''
        Function to imitate a given engagement type.
        '''
        center = (ad_size[0]/2, self.browser_edges[0] + (ad_size[1]/2))

        if self.engagement_type == "tap":
            pyautogui.moveTo(center[0], center[1], duration=1)
            pyautogui.leftClick()

        elif self.engagement_type == "swipe right":
            pyautogui.moveTo(center[0], center[1], duration=1)
            pyautogui.dragRel(center[0], 0, duration=1)

        elif self.engagement_type == "swipe left":
            pyautogui.moveTo(center[0], center[1], duration=1)
            pyautogui.dragRel(-center[0], 0, duration=1)

        elif self.engagement_type == "tap and hold":
            pyautogui.moveTo(center[0], center[1], duration=1)
            pyautogui.click()

        elif self.engagement_type == "scrub":
            pyautogui.moveTo(center[0] - (1/2 * center[0]),
                             center[1] - (2/3 * center[1]), duration=0.2)
            pyautogui.dragRel(center[0], 0, duration=0.2)
            pyautogui.dragRel(-center[0], (1/3 * center[1]), duration=0.2)
            pyautogui.dragRel(center[0], 0, duration=0.2)
            pyautogui.dragRel(-center[0], (1/3 * center[1]), duration=0.2)
            pyautogui.dragRel(center[0], 0, duration=0.2)
            
    def generate_preview_video(self) -> None:
        '''
        Function to generate preview video and also a cropped version of the video.
        '''
        # Initialize Selenium WebDriver

        # driver = webdriver.Chrome(
        #     options=self.opt, desired_capabilities=self.capabilities)
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

        # Maximize WebDriver's Window to Maximum Size
        driver.maximize_window()

        try:

            # Load AD-Unit through Selenium
            driver.get(self.preview_url)

            # Locate AD-Unit Element from Browser
            canvas = driver.find_element(By.TAG_NAME, "canvas")
            
            # # Start Recording Entire Screen
            # video_recording = Popen(self.cmd)
            video_recording = subprocess.Popen(self.cmd,shell=True,stdout=subprocess.PIPE,stderr=subprocess.STDOUT)


            # # Identify Size of AD-Unit
            ad_size = (canvas.size.get("width"), canvas.size.get("height"))

            # Engage Ad-Unit
            self._imitate_engagement(ad_size)

            # Continuously Check Status of AD-Unit using its console logs
            # until it reached a "GAME COMPLETE" Status
            WebDriverWait(driver, 100).until(self.is_status_complete)

            sleep(5)

            # Stop Video Recording
            self.terminate(video_recording)

            # Close Selenium Browser Window
            driver.close()

            # Crop Generated Preview Video Recording
            # self.crop_video(self.video_name, x_pos=0, y_pos=70,width=ad_size[0], height=ad_size[1])
            print(self.video_name)
            self.crop_video('./extracted_images/video', x_pos=0, y_pos=70,width=ad_size[0], height=ad_size[1])

        except TimeoutException:
            print("TimeOut Exception Fired")
            print("AD-Unit Status Console Logs did not Complete. Engagement Failed.")
            driver.close()

        except NoSuchElementException:
            print(f"AD-Unit Failed to Load: {self.preview_url}")
            driver.close()

    def generate_frames(self) -> None:
        '''
        Function to generate creative start and end frames.
        '''
        # Initialize Selenium WebDriver
        # driver = webdriver.Chrome(
        #     options=self.opt, desired_capabilities=self.capabilities, )
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

        # Maximize WebDriver's Window to Maximum Size
        driver.maximize_window()

        try:
            # Load AD-Unit through Selenium
            driver.get(self.preview_url)

            # Locate AD-Unit Element from Browser
            canvas = driver.find_element(By.TAG_NAME, "canvas")

            # Capture Start Frame
            canvas.screenshot(
                path.join(self.save_location, f'{self.file_name}_start_frame.png'))
            print('Start Frame captured')

            # Identify Size of AD-Unit
            ad_size = (canvas.size.get("width"), canvas.size.get("height"))

            # Engage Ad-Unit
            self._imitate_engagement(ad_size)

            # Continuously Check Status of AD-Unit using its console logs
            # until it reached a "GAME COMPLETE" Status
            WebDriverWait(driver, 100).until(self.is_status_complete)

            sleep(5)

            # Capture End Frame
            canvas.screenshot(path.join(self.save_location,f'{self.file_name}_end_frame.png'))
            print('End Frame Captured')

            # Close Selenium Browser Window
            driver.close()

        except TimeoutException:
            print("TimeOut Exception Fired")
            print("AD-Unit Status Console Logs did not Complete. Engagement Failed.")
            driver.close()

        except NoSuchElementException:
            print(f"AD-Unit Failed to Load: {self.preview_url}")
            driver.close()


In [4]:
# Creating Extractor Object
ext = CreativeFrameExtractor(
    'https://s3.us-east-1.amazonaws.com/a.futureadlabs.com-us-east-1-backup/us-east-1/games/5a48ffcbf611f167ecbb884e807f31f6/7e6dcb347f24c7843a8d/index.html', 'tap', save_location='./extracted_images/')

./extracted_images/5a48ffcbf611f167ecbb884e807f31f6-7e6dcb347f24c7843a8d
5a48ffcbf611f167ecbb884e807f31f6-7e6dcb347f24c7843a8d
./extracted_images/


In [5]:
# Calling Generate Frames from Extractor Object
ext.generate_frames()

Start Frame captured
Starting Recording AD-UNIT...
{'level': 'INFO', 'message': 'https://s3.us-east-1.amazonaws.com/a.futureadlabs.com-us-east-1-backup/us-east-1/games/5a48ffcbf611f167ecbb884e807f31f6/7e6dcb347f24c7843a8d/index.html 19:16 "GAME CREATED"', 'source': 'console-api', 'timestamp': 1667529029909}
Stopped Recording AD-UNIT...
{'level': 'INFO', 'message': 'https://s3.us-east-1.amazonaws.com/a.futureadlabs.com-us-east-1-backup/us-east-1/games/5a48ffcbf611f167ecbb884e807f31f6/7e6dcb347f24c7843a8d/index.html 22:16 "GAME COMPLETE"', 'source': 'console-api', 'timestamp': 1667529047825}
End Frame Captured


In [6]:
# Calling Generate Preview Video from Extractor Object
ext.generate_preview_video()


Starting Recording AD-UNIT...
{'level': 'INFO', 'message': 'https://s3.us-east-1.amazonaws.com/a.futureadlabs.com-us-east-1-backup/us-east-1/games/5a48ffcbf611f167ecbb884e807f31f6/7e6dcb347f24c7843a8d/index.html 19:16 "GAME CREATED"', 'source': 'console-api', 'timestamp': 1667529060479}
Stopped Recording AD-UNIT...
{'level': 'INFO', 'message': 'https://s3.us-east-1.amazonaws.com/a.futureadlabs.com-us-east-1-backup/us-east-1/games/5a48ffcbf611f167ecbb884e807f31f6/7e6dcb347f24c7843a8d/index.html 22:16 "GAME COMPLETE"', 'source': 'console-api', 'timestamp': 1667529078033}
./extracted_images/5a48ffcbf611f167ecbb884e807f31f6-7e6dcb347f24c7843a8d
./extracted_images/video


AttributeError: module 'ffmpeg' has no attribute 'input'

#### Extracting Colors

In [ ]:
import extcolors
import pandas as pd
from colormap import rgb2hex
from matplotlib import pyplot as plt
from PIL import Image

ModuleNotFoundError: No module named 'extcolors'

In [ ]:
def identify_color_composition(image,
                               tolerance: int = 12,
                               limit: int = 1,
                               visualize: bool = False) -> None:
    """Function that identifies the color composition of a
    given image path."""

    extracted_colors = extcolors.extract_from_path(
        image, tolerance=tolerance, limit=limit)

    identified_colors = color_to_df(extracted_colors)

    if not visualize:
        return identified_colors

    list_color = list(identified_colors['c_code'])
    list_percent = [int(i) for i in list(identified_colors['occurrence'])]

    text_c = [c + ' ' + str(round(p*100/sum(list_percent), 1)) + '%' for c, p in zip(list_color,
                                                                                     list_percent)]
    fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(100, 100), dpi=10)
    wedges, _ = ax[0].pie(list_percent,
                          labels=text_c,
                          labeldistance=1.05,
                          colors=list_color,
                          textprops={'fontsize': 60, 'color': 'black'}
                          )

    plt.setp(wedges, width=0.3)

    # create space in the center
    plt.setp(wedges, width=0.36)

    ax[0].set_aspect("equal")
    fig.set_facecolor('grey')

    ax[1].imshow(Image.open(image))

    plt.show()

    return identified_colors


def color_to_df(extracted_colors: tuple):
    """Converts RGB Color values from extcolors output to HEX Values."""

    colors_pre_list = str(extracted_colors).replace(
        '([(', '').replace(')],', '), (').split(', (')[0:-1]
    df_rgb = [i.split('), ')[0] + ')' for i in colors_pre_list]
    df_percent = [i.split('), ')[1].replace(')', '')
                  for i in colors_pre_list]

    # convert RGB to HEX code
    df_rgb_values = [(int(i.split(", ")[0].replace("(", "")),
                      int(i.split(", ")[1]),
                      int(i.split(", ")[2].replace(")", ""))) for i in df_rgb]

    df_color_up = [rgb2hex(int(i.split(", ")[0].replace("(", "")),
                           int(i.split(", ")[1]),
                           int(i.split(", ")[2].replace(")", ""))) for i in df_rgb]

    colors_df = pd.DataFrame(zip(df_color_up, df_rgb_values, df_percent),
                             columns=['c_code', 'rgb', 'occurrence'])

    return colors_df


In [ ]:
identify_color_composition('./extracted_images/5a48ffcbf611f167ecbb884e807f31f6-7e6dcb347f24c7843a8d_start_frame.png',tolerance=100,limit=8, visualize=True)

In [ ]:
identify_color_composition(
    './Challenge_Data/Assets/5a48ffcbf611f167ecbb884e807f31f6/engagement_instruction.png', visualize=True)

In [ ]:
identify_color_composition(
    './Challenge_Data/Assets/5a48ffcbf611f167ecbb884e807f31f6/cta.png', limit=3 ,visualize=True)

> What features can be Extracted from Color ?
- Number of Colors used
- Background Color
- Text Color
- Contrast, Saturation, ...
- and more ...

#### Extracting Text

In [ ]:
import pytesseract
import cv2
import numpy as np
from typing import List, Tuple

In [ ]:
def convert_hex_to_rgb(hex_color: str, normalize: bool = True) -> List[str]:
    """Converts a HEX color to a RGB color

    Args:
        hex_color (str): HEX color code to convert
        normalize (bool, optional): Choice to normalize calculated rgb values . Defaults to True.

    Returns:
        List[str]: List of RGB values in order, normalized or not.
    """
    colors = hex_color[1:]

    # Convert HEX color values to RGB Values
    colors = [int(colors[0:2], base=16),  # RED
              int(colors[2:4], base=16),  # GREEN
              int(colors[4:6], base=16)]  # BLUE

    # Normalize RGB values
    if normalize:
        colors = [color / 255 for color in colors]

    return colors

def get_luminance(hex_color: str) -> float:
    """Calculates the luminance of a given HEX color

    Args:
        hex_color (str): HEX color code to calculate luminance for

    Returns:
        float: luminance value of color
    """
    colors = convert_hex_to_rgb(hex_color)

    luminance = colors[0] * 0.2126 + colors[1] * 0.7152 + colors[2] * 0.0722

    return luminance

def fix_image_background(image_path: str):
    identified_colors = identify_color_composition(image_path)
    text_color = identified_colors['c_code'].to_list()[0]
    text_color_rgb = identified_colors['rgb'].to_list()[0]
    luminance = get_luminance(hex_color=text_color)

    if luminance < 140:
        background_color = (255, 255, 255)
    else:
        background_color = (0, 0, 0)

    # Load image
    image = cv2.imread(image_path)

    # Make all perfectly green pixels white
    image[np.all(image != text_color_rgb, axis=-1)] = background_color

    return image

def extract_text(image_path, tesseract_cmd: str, fix_background: bool = False):
    pytesseract.pytesseract.tesseract_cmd = tesseract_cmd
    try:
        if fix_background:
            text = pytesseract.image_to_string(
                fix_image_background(image_path))
        else:
            text = pytesseract.image_to_string(image_path)

        return text

    except pytesseract.TesseractNotFoundError:
        raise Exception(
            f'Failure: Tesseract is not installed or not available in the defined path {tesseract_cmd}')

In [ ]:
extract_text('./extracted_images/5a48ffcbf611f167ecbb884e807f31f6-7e6dcb347f24c7843a8d_start_frame.png',
             r'C:\Program Files\Tesseract-OCR\tesseract')

In [ ]:
extract_text('./Challenge_Data/Assets/5a48ffcbf611f167ecbb884e807f31f6/engagement_instruction.png',
             r'C:\Program Files\Tesseract-OCR\tesseract', fix_background=True)

In [ ]:
extract_text('./Challenge_Data/Assets/5a48ffcbf611f167ecbb884e807f31f6/cta.png',
             r'C:\Program Files\Tesseract-OCR\tesseract', fix_background=True)

> What features can be Extracted from Text ?
- Number of Characters
- Number of Words
- ...

#### Locating Assets within Creative

In [ ]:
def locate_image_on_image(locate_image: str, on_image: str, prefix: str = '', visualize: bool = False, color: Tuple[int, int, int] = (0, 0, 255)):
    try:

        image = cv2.imread(on_image)
        gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

        template = cv2.imread(locate_image, 0)

        result = cv2.matchTemplate(gray, template, cv2.TM_CCOEFF)
        _, _, _, max_loc = cv2.minMaxLoc(result)

        height, width = template.shape[:2]

        top_left = max_loc
        bottom_right = (top_left[0] + width, top_left[1] + height)

        if visualize:
            cv2.rectangle(image, top_left, bottom_right, color, 1)
            plt.figure(figsize=(10, 10))
            plt.axis('off')
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            plt.imshow(image)

        return {f'{prefix}top_left_pos': top_left, f'{prefix}bottom_right_pos': bottom_right}

    except cv2.error as err:
        print(err)

In [ ]:
locate_image_on_image(
    './Challenge_Data/Assets/4c3bb41d4f40f39842b7b8d3f536366a/engagement_instruction.png', './Challenge_Data/Assets/4c3bb41d4f40f39842b7b8d3f536366a/_preview.png', prefix='eng_', visualize=True)


In [ ]:
locate_image_on_image(
    './Challenge_Data/Assets/fef95c5e1ee5bc235b56d7c508d3bcd0/engagement_instruction.png', './Challenge_Data/Assets/fef95c5e1ee5bc235b56d7c508d3bcd0/_preview.png', prefix='eng_', visualize=True)

> What features can be Extracted from Locating Assets ?
- Size
- Ratio
- Total Covered Area
- and more ...

#### Object Detection

In [ ]:
import torch

In [ ]:
%cd yolov7
! python detect.py --weights yolov7-tiny.pt --conf 0.4 --img-size 1280 --source ../extracted_images/5a48ffcbf611f167ecbb884e807f31f6-7e6dcb347f24c7843a8d_start_frame.png --project trial --name run
%cd ..

> What features can be Extracted from Object Detection ?
- Object Detected
- Number of Objects Detected
- and more ...